---
# Section 2.6: Propagation of Roundoff Errors in Gaussian Elimination
---

The $n \times n$ Hilbert matrix $H$ is defined by
$$h_{ij} = \frac{1}{i+j-1}, \quad i,j = 1,\ldots,n.$$
This matrix is known to be very ill-conditioned.

Catastrophic cancellation during Gaussian elimination can occur on such badly conditioned matrices.

In [ ]:
n = 7
H = Rational{BigInt}[1//(i+j-1) for i=1:n, j=1:n]

The inverse of $H$ is integer.

In [ ]:
inv(H)

In [ ]:
invH = round.(Int, inv(H))

In [ ]:
invH*H

In [ ]:
round.(Int, invH*H)

We will first compute the $LU$-decomposition of $H$ using fractional arithmetic to find the _exact_ values of $U$.

In [ ]:
using LinearAlgebra

In [ ]:
L, U, p = lu(H);
U

Now let's store the Hilbert matrix $H$ in double floating-point format.

In [ ]:
Hd = map(Float64, H)

The following $LU$-decomposition is computed using floating-point arithmetic.

In [ ]:
Ld, Ud, pd = lu(Hd);
Ud

Comparing the true $U$ with the $U_d$ that was computed using floating point arithmetic, we see that there a complete loss of precision.

In [ ]:
map(Float64, U)

In [ ]:
norm(Ud - map(Float64,U))/norm(map(Float64, U))

However, the error in the $LU$-decomposition of $H$ is small.

In [ ]:
E = Ld*Ud - Hd[pd,:]

In [ ]:
norm(E)/norm(Hd)

# Solving $Hx = b$

In [ ]:
n = 16
H = Rational{BigInt}[1//(i+j-1) for i=1:n, j=1:n]
x = ones(Rational{BigInt}, n)
b = H*x
xhat = H\b

In [ ]:
n = 16
H = Float64[1/(i+j-1) for i=1:n, j=1:n]
x = ones(n)
b = H*x
xhat = H\b

The computed solution $\hat{x}$ is very far from the true solution $x$.

In [ ]:
norm(xhat - x)/norm(x)

The Hilbert matrix is very ill-conditioned, so solving $Hx=b$ is very sensitive to roundoff errors. Here we compute the condition number $\kappa_2(H)$.

In [ ]:
cond(H)

However, the residual $\hat{r} = b - H\hat{x}$ is very small, so $\hat{x}$ exactly satisfies the slightly perturbed system

$$H\hat{x} = b + \delta b,$$

where $\delta b = -\hat{r}$.

In [ ]:
rhat = b - H*xhat;
norm(rhat)/norm(b)

Therefore, the computation of $\hat{x}$ is **backward stable**. The large error in $\hat{x}$ is completely due to the fact that $H$ is very ill-conditioned.

In [ ]:
cond(H)*norm(rhat)/norm(b)

---